In [1]:
from src.utils import save, load
import numpy as np

/home/steinad/anaconda3/envs/lexx/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
roberta_topic_vals = load("topic_vals_roberta_sst2").reshape(872, 73)
roberta_word_vals = load("word_vals_roberta_sst2")
gpt2_topic_vals = load("topic_vals_gpt2_sst2").reshape(872, 73)
gpt2_word_vals = load("word_vals_gpt2_sst2")

In [6]:
print(roberta_word_vals[0])

[0.017804070766314323, 0.017820183888618602, 0.009020082382134824, 0.00904148044960919, 0.008973233131341367, 0.008827978740761052, 0.00885760224921442, 0.008831167589256535, 0.00884090301456985, 0.0005314120879540077]


In [22]:
diffs = np.sum(np.abs(roberta_topic_vals - gpt2_topic_vals), axis=1)
biggest_diff = np.argsort(diffs)
print(diffs[biggest_diff[-5:]])

[3.43656601 3.46910591 3.60732746 4.01513563 4.80082569]


In [17]:
from datasets import load_dataset

sst2 = load_dataset("sst2", split="validation")
x = [sst2[i]['sentence'] for i in range(len(sst2))]

Found cached dataset sst2 (/home/steinad/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5)


In [8]:
from src.pair_data import LIWCWordData

def get_liwc_topics():
    feature_groups = LIWCWordData(
        "../data/LIWC2015_processed.csv",
        split="train")

    print(feature_groups.groups)
    topic_names = feature_groups.groups
    topics = {name: set() for name in topic_names}
    for word, groups in feature_groups:
        for group in groups:
            topics[group].add(word)
    all_tokens = set().union(*topics.values())
    word2idx = {word: i for i, word in enumerate(all_tokens)}
    topics = np.array([[1.0 if tok in topics[topic_names[i]]
                        else 0.0 for tok in all_tokens] for i in range(len(topic_names))])
    topics = topics / np.sum(topics, axis=1, keepdims=True)
    return topics, topic_names, word2idx

topics, topic_names, word2idx = get_liwc_topics()

['ACHIEVE' 'ADJ' 'ADVERB' 'AFFECT' 'AFFILIATION' 'ANGER' 'ANX' 'ARTICLE'
 'ASSENT' 'AUXVERB' 'BIO' 'BODY' 'CAUSE' 'CERTAIN' 'COGPROC' 'COMPARE'
 'CONJ' 'DEATH' 'DIFFER' 'DISCREP' 'DRIVES' 'FAMILY' 'FEEL' 'FEMALE'
 'FILLER' 'FOCUSFUTURE' 'FOCUSPAST' 'FOCUSPRESENT' 'FRIEND' 'FUNCTION'
 'HEALTH' 'HEAR' 'HOME' 'I' 'INFORMAL' 'INGEST' 'INSIGHT' 'INTERROG'
 'IPRON' 'LEISURE' 'MALE' 'MONEY' 'MOTION' 'NEGATE' 'NEGEMO' 'NETSPEAK'
 'NONFLU' 'NUMBER' 'PERCEPT' 'POSEMO' 'POWER' 'PPRON' 'PREP' 'PRONOUN'
 'QUANT' 'RELATIV' 'RELIG' 'REWARD' 'RISK' 'SAD' 'SEE' 'SEXUAL' 'SHEHE'
 'SOCIAL' 'SPACE' 'SWEAR' 'TENTAT' 'THEY' 'TIME' 'VERB' 'WE' 'WORK' 'YOU']


In [26]:
def sort_shap(shap_values, feature_names):
    sort_idx = np.argsort(shap_values)
    return shap_values[sort_idx], feature_names[sort_idx]

print(x[biggest_diff[-1]])
topic_values, sorted_names = sort_shap(roberta_topic_vals[biggest_diff[-1]], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

print()
topic_values, sorted_names = sort_shap(gpt2_topic_vals[biggest_diff[-1]], topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")
print(np.sum(topic_values))

it 's hard to like a film about a guy who is utterly unlikeable , and shiner , starring michael caine as an aging british boxing promoter desperate for a taste of fame and fortune , is certainly that . 
LEISURE: -0.36569334224242755
FUNCTION: -0.27105720183691845
MONEY: -0.21411185965103244
ADJ: -0.18873733102676638
PERCEPT: -0.18630983465633832
FOCUSPRESENT: 0.09921325059868341
VERB: 0.09921325059868341
AUXVERB: 0.09921325059868341
COGPROC: 0.9411406235419841
TENTAT: 0.9411406235419841
-0.01598477363586448

PERCEPT: -0.14441831652159903
ADJ: -0.13117476043623966
FEEL: -0.12668385165530482
VERB: -0.04428620531400239
AUXVERB: -0.04428620531400239
RELATIV: 0.032976173363023935
ADVERB: 0.03655655451021553
FUNCTION: 0.12921181384431246
ARTICLE: 0.15489520962575293
LEISURE: 0.27505445752494306
-0.039242565631866455


In [25]:
roberta_feat_imp = np.sum(np.abs(roberta_topic_vals), axis=0)
gpt2_feat_imp = np.sum(np.abs(gpt2_topic_vals), axis=0)

topic_values, sorted_names = sort_shap(roberta_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")

print()
topic_values, sorted_names = sort_shap(gpt2_feat_imp, topic_names)
for i in range(5):
    print(f"{sorted_names[i]}: {topic_values[i]}")
for i in range(5, 0, -1):
    print(f"{sorted_names[-i]}: {topic_values[-i]}")

FILLER: 0.0
FRIEND: 0.03189607823355886
NONFLU: 0.033081506995165516
NETSPEAK: 0.06012690665245049
FAMILY: 0.07551659329808283
VERB: 15.59652999899614
AFFECT: 18.68109704727057
FUNCTION: 56.65816143713713
TENTAT: 139.1511898138908
COGPROC: 146.15578183325945

FILLER: 0.0
NETSPEAK: 0.03222749590764896
FRIEND: 0.04757091170766162
NONFLU: 0.0950164186725323
WE: 0.1060787855539368
ADJ: 14.420973437055986
COGPROC: 16.749330594025174
AFFECT: 20.280769169058956
FUNCTION: 46.39188136829717
LEISURE: 262.2904466820398
